## Download the necessary libraries
- Numpy: for array creation, inv, transpose and matrix multiplication
- math: here for exp(used in sigmoid)

In [31]:
import numpy as np
import math

**Question 1:** Calculate Covariance Matrix

Given a matrix where each vector represents a feature i.e. same feature value for different data points, calculate the covariance matrix.

A covariance matrix is a feature * feature matrix to represent how separate features interact with each other.



Steps:
- subtract feature means from values.
- for each pair of features
    - calculate the elementwise product.
    - sum all the elementwise product.
    - divide by number of data points.
    
**NOTE:** I will write two solutions for this problem, using numpy and without using numpy to provide a bridge to understand parallelization

In [17]:
# without using numpy
def calculate_covariance_matrix(vectors: list[list[float]]) -> list[list[float]]:
    means=[sum(feature_vector)/len(feature_vector) for feature_vector in vectors]
    vectors=[[feature_value-means[i] for feature_value in feature_vector] for i,feature_vector in enumerate(vectors)]
    covariance_matrix=[]
    for i,vec1 in enumerate(vectors):
        covariance_matrix.append([sum([vec1[k]*val for k,val in enumerate(vec2)])/(len(vectors[0])-1) for j,vec2 in enumerate(vectors)])
    return covariance_matrix

# using numpy
def calculate_covariance_matrix_numpy(vectors: list[list[float]]) -> list[list[float]]:
    vectors=np.array(vectors)
    features,data_points=vectors.shape
    means=np.mean(vectors,axis=1).reshape(-1,1)
    vectors=vectors-means
    covariance_matrix=vectors @ vectors.T
    covariance_matrix/=(data_points-1)
    return covariance_matrix.tolist()

**Question 2:** Solve linear equations using Jacobi algorithm

Jacobi Algorithm is an iterative algorithm to solve linear equations
Given a square matrix A(n equations, n unknowns) and a vector b, iteratively change an initialized vector x, so that

$Ax=b$

Algorithm
1. Initialize x
2. for each of n iterations update
     
     $x_i$ = (1/$A_i$$_i$) * ( $b_i$ - sum($A_i$$_j$ * $x_j$))

**NOTE:**
1. $b_i$ - sum($A_i$$_j$ * $x_j$) is basically b-Ax
2. Although the method states rounding at every iteration, all test cases are only accepted if rounding is done at the end.

In [ ]:
def solve_jacobi(A: np.ndarray, b: np.ndarray, n: int) -> list:
    diag_a=np.diag(A)
    non_diag_a=A-np.diag(diag_a)
    x=np.zeros(len(b))
    x_temp=np.zeros(len(b))
    for _ in range(n):
        x_temp = (b - (non_diag_a @ x)) / diag_a
        x=x_temp.copy()
    return np.round(x,4).tolist()

**Question 3:** K-Means Clustering

Implement the K-Means algorithm for clustering given a set of points and a set of initial centroids. Run for a given number of iterations or until the clusters converge, whichever is faster.

Steps:
1. for each iteration
    - assign each point to a centroid point based on closest distance.
    - use each new group to create a new centroid point which is a mean of all the points in the group
    - check to see if the new centroid points are same as the old ones
        - If yes, stop the loop and return the centroid points
        - else, continue on to the next iteration

**NOTE:** 
1. There is an error in the parameter type as it only incorporates 2 dimensional points, whereas the code is tested on 3 dimensional points as well. Make sure to write a dimension agnostic code.
2. I am adding two solution (with/without numpy) for learning.

In [91]:
    
def k_means_clustering(points: list[tuple[float, float]], k: int, initial_centroids: list[tuple[float, float]], max_iterations: int) -> list[tuple[float, float]]:
    groups=dict()
    for _ in range(max_iterations):
        new_centroids=[None]*len(initial_centroids)
        for i,point in enumerate(points):
            group_index=np.argmin([(sum([(x1-x2)**2 for x1,x2 in zip(point,centroid)]))**0.5 for centroid in initial_centroids])
            groups[group_index]=groups.get(group_index,[])+[point]
        for index,point_list in groups.items():
            new_point=[0 for i in range(len(points[0]))]
            for point in point_list:
                for i in range(len(point)):
                    new_point[i]+=point[i]
            new_point=[round(val/len(point_list),4) for val in new_point]
            new_centroids[index]=new_point
        if new_centroids==initial_centroids:
            break
        initial_centroids=new_centroids
    final_centroids=initial_centroids
    return final_centroids

def k_means_clustering_numpy(points,k,initial_centroids,max_iterations):
    points=np.array(points)
    centroids=np.array(initial_centroids)
    
    for _ in range(max_iterations):
        distances= np.array([np.sqrt((points-centroid)**2).sum(axis=1) for centroid in centroids])
        groups=np.argmin(distances,axis=0)
        new_centroids= np.array([points[groups==group].mean(axis=0) if len(points[groups==group])>0 else centroids[i] for group in range(k)])
        
        if np.all(centroids==new_centroids):
            break
        centroids=new_centroids
        centroids=np.round(centroids,4)
    return [tuple(centroid) for centroid in centroids]


**Question 4:** Cross-Validation Data Split Implementation

Given a dataset, implement a k-fold cross validation split and return the train test split datasets.

**Mistakes to remember:**
1. np.concatenate takes a list/tuple of arguments. Do not give independent parameters.
2. During the kth iteration, make sure to take all the residual samples as well.

In [134]:
def cross_validation_split(data: np.ndarray, k: int) -> list:
    fold_size=len(data)//k
    folds=[]
    for i in range(k):
        start=i*fold_size
        end=(i+1)*fold_size if i<k-1 else len(data)
        test_data=data[start:end].tolist()
        train_data=np.concatenate((data[:start],data[end:])).tolist()
        folds.append([train_data,test_data])
    return folds